- save and load as  classmethod?
- overload constructor?

In [1]:
with open('datasets/shakespeare.txt') as file:
    text = file.read()

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from unidecode import unidecode
from tokenizer.bpe_tokenizer import BytePairEncodingTokenizer

tokenizer = BytePairEncodingTokenizer(0)
tokenizer.load('tokenizer/trained_tokenizers/bpe.pkl')

pipeline = Pipeline([
    ('unicode_to_ascii', FunctionTransformer(unidecode)),
    ('encode', FunctionTransformer(tokenizer.encode))
])

pipeline.transform('lowest')

[75, 125, 68, 106]